In [12]:
import pandas as pd
import numpy as np
import seaborn as sns

In [13]:
places = pd.read_pickle('data/places_preprocessed.pkl')
users = pd.read_pickle('data/users_preprocessed.pkl')
accidents = pd.read_pickle('data/accidents_preprocessed.pkl')

In [14]:
places.head()

,accident_id,road_category,road_regime,no_lanes,reserved_lane,road_gradient,road_plan,road_condition,infrastructure,accident_situation
0,201600000001,Departmental Road,Bidirectional,NaN,NaN,Dish,Curved right,normal,NaN,On the road
1,201600000002,Departmental Road,One way,NaN,NaN,Dish,Curved on the left,normal,NaN,On the road
2,201600000003,Departmental Road,Bidirectional,2.0,NaN,Dish,Curved right,wet,NaN,On the verge
3,201600000004,Communal Way,Bidirectional,NaN,NaN,Dish,Straight part,normal,NaN,On the road
4,201600000005,Communal Way,NaN,NaN,NaN,NaN,Straight part,normal,NaN,On the road


In [15]:
users.head()

,user_id,accident_id,place_in_car,user_type,injury_type,sex,equipment_used,pedestrian_action,pedestrian_alone,year_of_birth,vechicle_number
0,0,201600000001,1,Driver,Unscathed,Female,yes,not specified or not applicable,NaN,1983,B02
1,1,201600000001,1,Driver,Hospitalized wounded,Male,yes,not specified or not applicable,NaN,2001,A01
2,2,201600000002,1,Driver,Hospitalized wounded,Male,yes,not specified or not applicable,NaN,1960,A01
3,3,201600000002,2,Passenger,Hospitalized wounded,Male,yes,not specified or not applicable,NaN,2000,A01
4,4,201600000002,3,Passenger,Hospitalized wounded,Female,yes,not specified or not applicable,NaN,1962,A01


In [16]:
accidents.head()

,accident_id,lighting,localization,intersection_type,weather,collision_type,com,address,gps,lat,long,departament,time,year,month,hour
0,201600000001,Full day,In built-up areas,Out of intersection,Cloudy weather,Two vehicles - by the side,5,"46, rue Sonneville",Métropole,NaN,NaN,590,2016-02-16 14:45:00,2016.0,2.0,14.0
1,201600000002,Full day,In built-up areas,Giratory,Normal,Other collision,5,1a rue du cimetière,Métropole,NaN,NaN,590,2016-03-16 18:00:00,2016.0,3.0,18.0
2,201600000003,Full day,Out of agglomeration,Out of intersection,Normal,Other collision,11,NaN,Métropole,NaN,NaN,590,2016-07-16 19:00:00,2016.0,7.0,19.0
3,201600000004,Twilight or dawn,In built-up areas,Out of intersection,Dazzling weather,Two vehicles - by the side,477,52 rue victor hugo,Métropole,NaN,NaN,590,2016-08-16 19:30:00,2016.0,8.0,19.0
4,201600000005,Full day,In built-up areas,Intersection in T,Normal,Two vehicles - by the side,11,rue Joliot curie,Métropole,NaN,NaN,590,2016-12-16 11:00:00,2016.0,12.0,11.0


In [17]:
places.shape

(839985, 10)

Places and accidents are one-to-one relation. Joining

In [18]:
accidents.dtypes

accident_id                  object
lighting                     object
localization                 object
intersection_type            object
weather                      object
collision_type               object
com                          object
address                      object
gps                          object
lat                          object
long                         object
departament                  object
time                 datetime64[ns]
year                        float64
month                       float64
hour                        float64
dtype: object

In [19]:
master_table = accidents.set_index("accident_id").join(places.set_index("accident_id"))#, on = "accident_id")

Now joining users table - beware that for each accidents there can be multiple users.

In [20]:
master_table = master_table.join(users.set_index("accident_id"))

In [21]:
master_table = master_table.reset_index()

In [22]:
master_table['y'] = master_table.injury_type.map({"Unscathed": 0, 
                             "Light injury": 0,
                              "Hospitalized wounded": 1,
                              "Killed": 1
                             })

In [23]:
master_table.to_pickle('data/master_table.pkl')

In [24]:
with pd.option_context('display.max_rows', 20, 'display.max_columns', 50):
    print(master_table.head())

    accident_id                       lighting       localization  \
0  200500000001  Night without public lighting  In built-up areas   
1  200500000001  Night without public lighting  In built-up areas   
2  200500000001  Night without public lighting  In built-up areas   
3  200500000001  Night without public lighting  In built-up areas   
4  200500000001  Night without public lighting  In built-up areas   

     intersection_type weather              collision_type com address  \
0  Out of intersection  Normal  Two vehicles - by the side  11   CD41B   
1  Out of intersection  Normal  Two vehicles - by the side  11   CD41B   
2  Out of intersection  Normal  Two vehicles - by the side  11   CD41B   
3  Out of intersection  Normal  Two vehicles - by the side  11   CD41B   
4  Out of intersection  Normal  Two vehicles - by the side  11   CD41B   

         gps      lat    long departament                time    year  month  \
0  Métropole  5051500  294400         590 2005-01-05 19:00:0